In [1]:
#Import Libraries
import pandas as pd
import numpy as np

%matplotlib inline 

import matplotlib
import matplotlib.pyplot as plt 
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
from sklearn import metrics

Using TensorFlow backend.


## 1. Read the dataset

In [2]:
# reading the CSV file into pandas dataframe
Loan = pd.read_csv("C:/Users/Tanu/Desktop/Greatleaning/Feb'19/Project_Neuralnet/bank.csv")  

print('Getting the glimpse of the data')
Loan.head(5)

print('Dimensions')
Loan.shape

print('variable type')
Loan.info()


Getting the glimpse of the data
Dimensions
variable type
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [3]:
# To check if any missing value in the dataset

Loan.isnull().values.any()

# No missing value in the data

False

# 2. Drop the columns which are unique for all users like IDs 

In [4]:
# dropping id, date & zipcode columns  as these columns are unique for all user
Loan1 = Loan.drop({'RowNumber','CustomerId','Surname'}, axis=1)

# Seeing output
Loan1.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# 3. Distinguish the feature and target set 

In [5]:
# From the problem statement of the project, it is clear that bank want to predict if the person will stay or it will exit
# Bank want to build the classifier to make the prediction

# Basis this information, 'Exit' is the target variable which shows if the customer is stayed or leave. 
#Exit = 1 menas customer has left

#Rest of the variable from Loan1 dataframe can be used for feature set  

# The task of creating Feature set & target set is done below:

In [6]:
# creating the feature set 
X = Loan1.drop('Exited', axis = 1)


# creating the target set 
y = Loan1['Exited']

In [7]:
# Seeing output of X

X.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58


In [8]:
# From the above output, we can see that 'Geogarphy' & 'Gender' are categorical 
#we need to convert categorical variables into numerical vars so that we can process them in the model

In [9]:
# Converting 'Gender' into numeric

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
X['Gender'] = labelencoder.fit_transform(X.Gender)

X['Gender'] = X['Gender'].astype('int64')

In [10]:
# Converting 'Geography' into numeric
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
X['Geography'] = labelencoder.fit_transform(X.Geography)

In [11]:
# As 'Geography' has three labels, changing these labels by creating dummy variables for each label & storing them in specific columns 
# using one hot coding 

# Importing one-hot encoder
from sklearn.preprocessing import OneHotEncoder

# creating dummy columns 
countryhotencoder = OneHotEncoder(categorical_features = [1]) 
X = countryhotencoder.fit_transform(X).toarray()

C:\Users\Tanu\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\Tanu\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:385: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [12]:
# Shape
X.shape

(10000, 12)

In [13]:
# Dropping one label of geography so that multicolinerty won't occure
X = X[:,1:]

In [14]:
# Seeing output of y
y.head(5)

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

# 4. Divide the data set into Train and test sets

In [15]:
#Test train split into 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=201)


# 5. Normalize the train and test data 

In [16]:
# Scaling 
#Importing Standardscaler 
from sklearn.preprocessing import StandardScaler

# Standardizing 
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# 6. Initialize & build the model 

In [17]:
# Importing Keras & corresponding packages
import keras
from keras.models import Sequential 
from keras.layers import Dense

In [18]:
# Initializing Neural net & setting up the parameters for its layer

#Initializing Neural Network
classifier = Sequential()

# Creating the first hidden layer
classifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))

# Creating the second hidden layer
classifier.add(Dense(activation = 'relu', units=6, kernel_initializer='uniform')) 

#Creating output layer
classifier.add(Dense(activation = 'sigmoid', units=1, kernel_initializer='uniform')) 

# 7.Optimize the model 

In [19]:
#Compiling the ANN using 'adam' optimizer
classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [20]:
# Checking length of Test -train 
print(len(X_train))
print(len(y_train))

8000
8000


In [21]:
#Fitting the classifier 
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 1s 138us/step - loss: 0.4838 - acc: 0.7982
Epoch 2/100
8000/8000 [==============================] - 1s 103us/step - loss: 0.4206 - acc: 0.8025
Epoch 3/100
8000/8000 [==============================] - 1s 97us/step - loss: 0.4106 - acc: 0.8340
Epoch 4/100
8000/8000 [==============================] - 1s 96us/step - loss: 0.4047 - acc: 0.8356
Epoch 5/100
8000/8000 [==============================] - 1s 113us/step - loss: 0.4002 - acc: 0.8357
Epoch 6/100
8000/8000 [==============================] - 1s 146us/step - loss: 0.3984 - acc: 0.8362
Epoch 7/100
8000/8000 [==============================] - 1s 148us/step - loss: 0.3967 - acc: 0.8367
Epoch 8/100
8000/8000 [==============================] - 1s 121us/step - loss: 0.3949 - acc: 0.8381
Epoch 9/100
8000/8000 [==============================] - 1s 101us/step - loss: 0.3938 - acc: 0.8387
Epoch 10/100
8000/8000 [==============================] - 1s 115us/step - loss: 0.3931 - acc: 0.8377
E

8000/8000 [==============================] - 1s 101us/step - loss: 0.3317 - acc: 0.8634
Epoch 82/100
8000/8000 [==============================] - 1s 86us/step - loss: 0.3331 - acc: 0.8617
Epoch 83/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.3312 - acc: 0.8644
Epoch 84/100
8000/8000 [==============================] - 1s 82us/step - loss: 0.3329 - acc: 0.8646
Epoch 85/100
8000/8000 [==============================] - 1s 84us/step - loss: 0.3322 - acc: 0.8661
Epoch 86/100
8000/8000 [==============================] - 1s 82us/step - loss: 0.3316 - acc: 0.8637
Epoch 87/100
8000/8000 [==============================] - 1s 113us/step - loss: 0.3319 - acc: 0.8654
Epoch 88/100
8000/8000 [==============================] - 1s 101us/step - loss: 0.3320 - acc: 0.8635
Epoch 89/100
8000/8000 [==============================] - 1s 113us/step - loss: 0.3315 - acc: 0.8656
Epoch 90/100
8000/8000 [==============================] - 1s 111us/step - loss: 0.3322 - acc: 0.8632
Epoch 91

# 8.Predict the results using 0.5 as a threshold 

In [26]:
#Predicting the test results without any thresh hold
y_pred = classifier.predict(X_test)
print(y_pred)

#Predicting the test results with 5% threshhold
y_pred = (y_pred > 0.5)
print(y_pred1)

[[0.07451803]
 [0.22839832]
 [0.3333406 ]
 ...
 [0.06693814]
 [0.07258716]
 [0.01427286]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


# 9.Print confusion matrix  and Accuracy score

In [30]:
# Calculating confusion matrix

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1515   62]
 [ 244  179]]


In [31]:
# Accuracy Score
print ('Accuracy Score')
print (((cm[0][0]+cm[1][1])*100)/(cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0]))

Accuracy Score
84.7


In [ ]:
# From the above accuracy score, model is able to classify 84.7% observation correctly